In [1]:
import pandas as pd

import warnings
import os

warnings.filterwarnings('ignore')

Februari 2019

I nummer 6 2019 gjorde vi en grej på kommunernas och regionernas köp av verksamhet. Siffrorna är hämtade från Kolada och heter där för kommuner N00120 (föreningar) och N00119 (privata företag), och för regioner N60092 (föreningar) och N60088 (privata företag). Kroken är att SKL släpper i veckan en rapport i ämnet som heter _Köp av verksamhet 2017_. Resultatet i tidningen [kan man se här](https://dagenssamhalle.e-pages.pub/titles/dagenssamhalle/8599/publications/365/pages/2), och [webbpubliceringen hittar man här](https://dagenssamhalle.se/nyhet/stora-skillnader-i-kop-fran-privata-utforare-26128).



In [2]:
df = pd.read_excel('data/kommuner.xls')

In [3]:
def data_formatting(var,all_years=False):
    df = pd.read_excel(f'data/{var}.xls')
    
    # namn på värden att byta ut
    föreningar = df.loc[df.iloc[:,0].str.contains('förening')].iloc[0,0]
    företag = df.loc[df.iloc[:,0].str.contains('företag')].iloc[0,0]
    
    # kolumnformattering
    df.columns = ['vars',f'{var}','år','andel']
    df = df[[f'{var}','år','vars','andel']]
    
    df.loc[df['vars']==företag,'vars'] = 'företag'
    df.loc[df['vars']==föreningar,'vars'] = 'föreningar'
    
    max_år = str(df.år.max())
    
    # separat df för att beräkna andel för föreningar
    tmp = df.loc[df['år']==df.år.max()]

    tmp = tmp.pivot_table(index=f'{var}',columns='vars',values='andel')
    
    tmp[f'andel_förening_{max_år}']\
    = ((tmp['föreningar'] / tmp.sum(1))*100).round(1)

    tmp = tmp.reset_index()[[f'{var}',f'andel_förening_{max_år}']]
    
    # summerar företagandelar och föreningsandelar
    df = df.groupby([f'{var}','år']).sum().reset_index()

    df = df.pivot_table(index=f'{var}',columns='år').reset_index()

    df.columns = df.columns.droplevel(0)
    
    if not all_years:
        df = df.iloc[:,[0,1,-1]]
        df.columns = [f'{var}',df.columns[1],df.columns[-1]]
    else:
        df.columns = [f'{var}'] + df.columns[1:].tolist()
    
    # förändring i procentenheter
    df['förändring'] = df.iloc[:,-1] - df.iloc[:,1]

    df = df.sort_values(int(max_år),ascending=False).reset_index(drop=True)

    return df.merge(tmp,on=f'{var}',how='left')

In [4]:
regs = [x[:-4] for x in os.listdir('data')]

In [5]:
kompl = data_formatting(regs[0],all_years=True).sort_values('förändring',ascending=False)

In [6]:
kompl = kompl.set_index('regioner')

In [7]:
for col in kompl.columns[:-1]:
    kompl[col] = kompl[col].round(1)

In [8]:
os.makedirs('res')

In [9]:
kompl.reset_index().to_excel('res/alla_reg.xlsx',index=False)

In [10]:
for reg in regs:
    data_formatting(reg).to_excel(f'res/{reg}.xlsx',index=False)